In [1]:
from collections import Counter

import pandas as pd
import pybel
import pybel.struct
import requests
import pybel_tools.mutation
from pybel.io.jupyter import to_jupyter

In [2]:
emmaa_covid_graph = pybel.load('data/emmaa/covid19-indra.bel.nodelink.json')

In [3]:
emmaa_covid_graph.summarize()

covid19 vc794d7a3-2979-4c59-bde0-5ca7e8a7166a
Number of Nodes: 44540
Number of Edges: 128813
Number of Citations: 11601
Number of Authors: 0
Network Density: 6.49E-05
Number of Components: 1814
Number of Warnings: 0


In [4]:
# look at the first 5 nodes in the graph
for i, node in zip(range(5), emmaa_covid_graph.nodes()):
    print(node)

a(CHEBI:10022 ! deoxynivalenol)
a(CHEBI:10023 ! voriconazole)
a(CHEBI:100241 ! ciprofloxacin)
a(CHEBI:100246 ! norfloxacin)
a(CHEBI:10043 ! wogonin)


In [5]:
# Check a protein is in the graph
histone_h3_family = pybel.dsl.Protein(namespace='FPLX', identifier='Histone_H3', name='Histone_H3')
histone_h3_family in emmaa_covid_graph

True

In [6]:
# Get all proteins whose names are ACE2 - might have modifications, too!
ace2s = {
    node
    for node in emmaa_covid_graph
    if isinstance(node, pybel.dsl.BaseConcept) # This line means check if the node has a name
    if node.name == 'ACE2'
}

ace2s

{<BEL p(HGNC:13557 ! ACE2)>,
 <BEL p(HGNC:13557 ! ACE2, pmod(Glyco))>,
 <BEL p(HGNC:13557 ! ACE2, pmod(Ph))>,
 <BEL p(HGNC:13557 ! ACE2, pmod(Ph, Ser, 680))>,
 <BEL p(HGNC:13557 ! ACE2, pmod(Ub))>,
 <BEL p(HGNC:13557 ! ACE2, var("p.?"))>}

In [7]:
# get all proteins named angiotensin, turns out there's only 1
angiotensin = [ 
    node
    for node in emmaa_covid_graph
    if isinstance(node, pybel.dsl.BaseConcept)
    if node.name.lower() == 'angiotensin'
][0]
angiotensin

<BEL a(CHEBI:48433 ! angiotensin)>

In [8]:
# Get all ACE2-angiotensin edges
for u, v, data in emmaa_covid_graph.edges(ace2s, data=True):
    if v == angiotensin:
        print()
        print(emmaa_covid_graph.edge_to_bel(u, v, data))
        print(data['evidence'])


p(HGNC:13557 ! ACE2) increases act(a(CHEBI:48433 ! angiotensin))
In this study, the purified ACE2 was able to convert Ang II into Ang 1-7 (XREF_FIG A and B) (K m = 2.87 +/-0.76 muM, XREF_FIG), suggesting that ACE2 prevents Ang II accumulation in the glomeruli, by regulating the degradation of angiotensin peptides and supporting the concept that ACE2 acts in a conter-regulatory manner to ACE and may play an important role to modulate the balance between vasoconstrictors and vasodilators in the kidney.

p(HGNC:13557 ! ACE2) increases act(a(CHEBI:48433 ! angiotensin))
Upon its discovery , ACE-2 was shown to generate angiotensin 1-7 ( Ang ( 1-7 ) ) from Ang II , and , to a lesser extent , angiotensin 1-9 ( Ang ( 1-9 ) ) from Ang I [ 23 , 24 , 28 ] .

p(HGNC:13557 ! ACE2) increases act(a(CHEBI:48433 ! angiotensin))
Down-regulation of ACE2 causes excessive production of angiotensin (ANG) II by the related enzyme ACE with stimulation of ANG type 1a receptor (AT1R) and enhanced lung vascular 

In [9]:
ace2_graph = pybel.struct.get_subgraph_by_neighborhood(emmaa_covid_graph, ace2s)

# Make sure all the edges between are added
pybel_tools.mutation.expand_internal(emmaa_covid_graph, ace2_graph)

ace2_graph.summarize()

None vNone
Number of Nodes: 404
Number of Edges: 5140
Number of Citations: 1875
Number of Authors: 0
Network Density: 3.16E-02
Number of Components: 1
Number of Warnings: 0


In [10]:
pybel.struct.count_functions(ace2_graph)

Counter({'Protein': 92,
         'Abundance': 161,
         'BiologicalProcess': 45,
         'Complex': 106})

In [11]:
pybel.struct.count_namespaces(ace2_graph)

Counter({'HGNC': 217,
         'bel': 8,
         'FPLX': 27,
         'UP': 16,
         'CHEBI': 71,
         'MESH': 37,
         'TEXT': 134,
         'PFAM': 8,
         'PUBCHEM': 1,
         'GO': 20})

In [12]:
to_jupyter(ace2_graph)

<IPython.core.display.Javascript object>

In [ ]:
# Get all process-process relations in a dataframe

df = pd.DataFrame([
    (str(u), str(v))
    for u, v, data in emmaa_covid_graph.edges(data=True)
    if isinstance(u, pybel.dsl.BiologicalProcess) and isinstance(v, pybel.dsl.BiologicalProcess)
])
df.drop_duplicates().head()

,0,1
0,"bp(GO:""GO:0000077"" ! ""DNA damage checkpoint"")","bp(GO:""GO:0006915"" ! ""apoptotic process"")"
1,"bp(GO:""GO:0000266"" ! ""mitochondrial fission"")","bp(GO:""GO:0006096"" ! ""glycolytic process"")"
2,"bp(GO:""GO:0000266"" ! ""mitochondrial fission"")","bp(GO:""GO:0006915"" ! ""apoptotic process"")"
3,"bp(GO:""GO:0000423"" ! mitophagy)","bp(GO:""GO:0030154"" ! ""cell differentiation"")"
4,"bp(GO:""GO:0000423"" ! mitophagy)","bp(MESH:D002470 ! ""Cell Survival"")"
